# Crawl4AI with local LLMs

In [1]:
run "../src/utils.py"

In [1]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.7.6 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 0.38s 
[FETCH]... ↓ https://crawl4ai.com                                                                    
| ✓ | ⏱: 3.91s 
[SCRAPE].. ◆ https://crawl4ai.com                                                                    
| ✓ | ⏱: 0.02s 
[COMPLETE] ● https://crawl4ai.com                                                                    
| ✓ | ⏱: 3.93s 
[COMPLETE] ● ✅ Crawling test passed! 


# Basic web crawling

Provided in the docs:

In [ ]:
# If running in notebooks
nest_asyncio.apply()

In [87]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

async def main():
    browser_conf = BrowserConfig(headless=True)  # or False to see the browser
    run_conf = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS
    )

    async with AsyncWebCrawler(config=browser_conf) as crawler:
        result = await crawler.arun(
            url="https://example.com",
            config=run_conf
        )
        print(result.markdown)

if __name__ == "__main__":
    asyncio.run(main())

[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ https://example.com                                                                     
| ✓ | ⏱: 0.72s 
[SCRAPE].. ◆ https://example.com                                                                     
| ✓ | ⏱: 0.00s 
[COMPLETE] ● https://example.com                                                                     
| ✓ | ⏱: 0.72s 
# Example Domain
This domain is for use in documentation examples without needing permission. Avoid use in operations.
[Learn more](https://iana.org/domains/example)



## Use with local LLM

In [ ]:
# download your model in terminal
# ollama pull qwen2.5:3b

In [ ]:
import os
import asyncio
import json
from pydantic import BaseModel, Field
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, LLMConfig
from crawl4ai import LLMExtractionStrategy

class Product(BaseModel):
    name: str
    description: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/qwen2.5:3b", api_token=None),
        schema=Product.model_json_schema(),
        extraction_type="schema",
        instruction=""" 
        From the crawled content
        extract the titles and the description in JSON format like this:
        {"title": "title name", "description: "description text"}
        """,
        chunk_token_threshold=1000,
        overlap_rate=0.0,
        apply_chunking=True,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 500}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(headless=True)

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://www.scrapethissite.com/pages/",
            config=crawl_config
        )

        if result.success:
            # 5. The extracted content is presumably JSON
            data = json.loads(result.extracted_content)
            print("Extracted items:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)
        
        return data 

if __name__ == "__main__":
    asyncio.run(main())


[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ https://www.scrapethissite.com/pages/                              
| ✓ | ⏱: 1.12s 
[SCRAPE].. ◆ https://www.scrapethissite.com/pages/                              
| ✓ | ⏱: 0.00s 
[EXTRACT]. ■ https://www.scrapethissite.com/pages/                              
| ✓ | ⏱: 4.28s 
[COMPLETE] ● https://www.scrapethissite.com/pages/                              
| ✓ | ⏱: 5.41s 
Extracted items: [{'title': 'Countries of the World: A Simple Example', 'summary': 'A single page that lists information about all the countries in the world. Good for those just get started with web scraping.'}, {'title': 'Hockey Teams: Forms, Searching and Pagination', 'summary': 'Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.'}, {'title': 'Oscar Winning Films: AJAX and Javascript', 'summary': 'Click through a bunch of great films. Learn how content is added to the page asynchronously with

### "Real" website

Let's try with a real world website

In [ ]:

class Product(BaseModel):
    name: str
    price: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/qwen2.5:3b", api_token=None),
        schema=Product.model_json_schema(),
        extraction_type="schema",
        instruction=""" 
        From the crawled content
        extract the name and the price in JSON format like this:
        {"name": "product name", "price": "price value"}
        """,
        chunk_token_threshold=500,
        overlap_rate=0.0,
        apply_chunking=True,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 1000}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(
        headless=True,
        text_mode=True,
        light_mode=True
        )

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://www.microcenter.com/product/670842/intel-core-i7-14700k-raptor-lake-s-refresh-34ghz-twenty-core-lga-1700-boxed-processor-heatsink-not-included",
            config=crawl_config
            
        )

        if result.success:
            # 5. The extracted content is presumably JSON
            data = json.loads(result.extracted_content)
            print("Extracted item:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)

In [ ]:
if __name__ == "__main__":
    asyncio.run(main())

[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ 
https://www.microcenter.com/product/670842/intel...e-lga-1700-boxed-processor-heatsink-not-included  
| ✓ | ⏱: 0.74s 
[SCRAPE].. ◆ 
https://www.microcenter.com/product/670842/intel...e-lga-1700-boxed-processor-heatsink-not-included  
| ✓ | ⏱: 0.00s 
[EXTRACT]. ■ 
https://www.microcenter.com/product/670842/intel...e-lga-1700-boxed-processor-heatsink-not-included  
| ✓ | ⏱: 1.34s 
[COMPLETE] ● 
https://www.microcenter.com/product/670842/intel...e-lga-1700-boxed-processor-heatsink-not-included  
| ✓ | ⏱: 2.09s 
Extracted item: [{'name': 'Intel Core i7-14700K Raptor Lake-S Refresh 34GHz Twenty-Core LGA 1700 Boxed Processor Heatsink Not Included', 'price': '$299.99'}]

=== Token Usage Summary ===
Type                   Count
------------------------------
Completion                71
Prompt                   719
Total                    790

=== Usage History ===
Request #    Completion       Prompt        Total
------------------------------------

---

## Limitations with complex website

In [ ]:

class Product(BaseModel):
    name: str
    price: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/qwen2.5:3b", api_token=None),
        schema=Product.model_json_schema(),
        extraction_type="schema",
        instruction=""" 
        From the crawled content
        extract the name and the price in JSON format like this:
        {"name": "product name", "price": "price value"}
        """,
        chunk_token_threshold=500,
        overlap_rate=0.0,
        apply_chunking=True,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 1000}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(
        headless=True,
        text_mode=True,
        light_mode=True
        )

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://www.amazon.com/Bose-Cancelling-Wireless-Bluetooth-Headphones/dp/B07Q9MJKBV/ref=sr_1_1?sr=8-1",
            config=crawl_config
            
        )

        if result.success:
            # 5. The extracted content is presumably JSON
            data = json.loads(result.extracted_content)
            print("Extracted item:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)

if __name__ == "__main__":
    asyncio.run(main())

## Unstructured webstie

In [ ]:
import os
import asyncio
import json
from pydantic import BaseModel, Field
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, LLMConfig
from crawl4ai import LLMExtractionStrategy

class Product(BaseModel):
    summary: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/qwen2.5:3b", api_token=None),
        schema=Product.model_json_schema(),
        extraction_type="schema",
        instruction=""" 
        From the crawled content make a summary including the program description, courses and admision dates
        """,
        chunk_token_threshold=500,
        overlap_rate=0.0,
        apply_chunking=False,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 1000}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(
        headless=True,
        text_mode=True,
        light_mode=True
        )

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://extension.harvard.edu/academics/programs/computer-science-masters-degree-program/#program-overview",
            config=crawl_config
            
        )

        if result.success:
            # 5. The extracted content
            data = json.loads(result.extracted_content)
            print("Extracted item:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)

if __name__ == "__main__":
    asyncio.run(main())

Structured data extraction

In [12]:
import os
import asyncio
import json
from pydantic import BaseModel, Field
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, LLMConfig
from crawl4ai import LLMExtractionStrategy

class Product(BaseModel):
    program: str
    courses: str
    admission: str

async def main():
    # 1. Define the LLM extraction strategy
    llm_strategy = LLMExtractionStrategy(
        llm_config = LLMConfig(provider="ollama/qwen2.5:3b", api_token=None),
        schema=Product.model_json_schema(),
        extraction_type="schema",
        instruction=""" 
        From the crawled content
        extract the program overview, the courses and the admission sections in JSON format like this:
        {"program": "program overview", "courses": "courses", "admission":"admission"}
        """,
        chunk_token_threshold=500,
        overlap_rate=0.0,
        apply_chunking=False,
        input_format="markdown",   # or "html", "fit_markdown"
        extra_args={"temperature": 0.0, "max_tokens": 1000}
    )

    # 2. Build the crawler config
    crawl_config = CrawlerRunConfig(
        extraction_strategy=llm_strategy,
        cache_mode=CacheMode.BYPASS
    )

    # 3. Create a browser config if needed
    browser_cfg = BrowserConfig(
        headless=True,
        text_mode=True,
        light_mode=True
        )

    async with AsyncWebCrawler(config=browser_cfg) as crawler:
        # 4. Let's say we want to crawl a single page
        result = await crawler.arun(
            url="https://extension.harvard.edu/academics/programs/computer-science-masters-degree-program/#program-overview",
            config=crawl_config
            
        )

        if result.success:
            # 5. The extracted content
            data = json.loads(result.extracted_content)
            print("Extracted item:", data)

            # 6. Show usage stats
            llm_strategy.show_usage()  # prints token usage
        else:
            print("Error:", result.error_message)

if __name__ == "__main__":
    asyncio.run(main())

[INIT].... → Crawl4AI 0.7.6 
[FETCH]... ↓ 
https://extension.harvard.edu/academics/programs...science-masters-degree-progra
m/#program-overview  | ✓ | ⏱: 1.71s 
[SCRAPE].. ◆ 
https://extension.harvard.edu/academics/programs...science-masters-degree-progra
m/#program-overview  | ✓ | ⏱: 0.07s 
[EXTRACT]. ■ 
https://extension.harvard.edu/academics/programs...science-masters-degree-progra
m/#program-overview  | ✓ | ⏱: 5.15s 
[COMPLETE] ● 
https://extension.harvard.edu/academics/programs...science-masters-degree-progra
m/#program-overview  | ✓ | ⏱: 6.94s 
Extracted item: [{'program': "The program offers a Master's degree in Computer Science that can be pursued online, during evenings, or at your own pace. The program is designed for lifelong learners from high school to retirement age.", 'courses': 'Courses are available on various topics such as Artificial Intelligence, Cybersecurity, Data Science, and Programming. Students have the option to explore these courses in a variety of formats i

Extracted item: [{'program': "The program offers a Master's degree in Computer Science that can be pursued online, during evenings, or at your own pace. The program is designed for lifelong learners from high school to retirement age.", 'courses': 'Courses are available on various topics such as Artificial Intelligence, Cybersecurity, Data Science, and Programming. Students have the option to explore these courses in a variety of formats including online, evening classes, and self-paced learning.', 'admission': "To be admitted into the program, applicants should ideally possess a bachelor's degree from an accredited institution, proficiency in programming languages such as Java, Python, or C++, and some work experience in a technical field. Additionally, they must have excellent problem-solving skills, attention to detail, and critical thinking abilities.", 'error': False}]
